In [6]:
import numpy as np
import sympy as sp
from numpy import sin, cos, pi

from scipy.spatial.transform import Rotation as R

In [117]:
def DH_trans(DH, joint_val):

    d, theta, a, alpha = (0,0,0,0)

    if (DH[0] == 'r'):

        d, theta, a, alpha = (DH[1], joint_val, DH[2], DH[3])

    elif (DH[0] == 'p'):

        d, theta, a, alpha = (joint_val, DH[1], DH[2], DH[3])

    elif (DH[0] == 'f'):

        d, theta, a, alpha = (DH[1], DH[2], DH[3], DH[4])

    trans_mat = np.array([[cos(theta), -1*sin(theta)*cos(alpha), sin(theta)*sin(alpha),    a*cos(theta)],
                          [sin(theta), cos(theta)*cos(alpha),    -1*cos(theta)*sin(alpha), a*sin(theta)],
                          [0,          sin(alpha),               cos(alpha),               d           ],
                          [0,          0,                        0,                        1           ]])

    return trans_mat


def joint_transforms(DH_Params):

    transforms = []

    current_DOF = 0

    transforms.append(np.eye(4))

    for DH in DH_Params:

        if (DH[0] == 'r'):
            transforms.append(DH_trans(DH, 0.0))
            current_DOF = current_DOF + 1
        elif (DH[0] == 'p'):
            transforms.append(DH_trans(DH, 0))
            current_DOF = current_DOF + 1
        else:
            transforms.append(DH_trans(DH, 0.0))

    return transforms

def joint_frames(transforms):
         
         joint_frames = [transforms[0]]
 
         for trans in transforms[1:]:
 
             joint_frames.append(joint_frames[-1] @ trans)
 
         return joint_frames
def euler_angles(pose):
    if pose[0, 2] == 1:
        theta_1, theta_2, theta_3 = np.arctan2(pose[1, 0], pose[1, 1]), pi/2, 0
    elif pose[0, 2] == -1:
        theta_1, theta_2, theta_3 = -np.arctan2(pose[1, 0], pose[1, 1]), pi/2, 0
    else:
        theta_1 = np.arctan2(-pose[1, 2], pose[2, 2])
        theta_2 = np.arcsin(pose[0, 2])
        theta_3 = np.arctan2(-pose[0, 1], pose[0, 0])
    return (theta_1, theta_2, theta_3)
       
def rotation_matrix_from_euler_angles(euler_angles, sequence):
    """
    Return a transformation matrix for the given euler angles.
    
    :param euler_angles: are the euler angles.
    :type euler_angles: Matrix
    ...
    :param sequence: is the euler angles configuration.
    :type sequence: string 
    ...
    raise ValueError: if an invalid or unsupported euler angle sequence is received.
    ...
    :return: a transformation matrix for the given euler angles.
    """
    sequence = sequence.lower()
    if not all([char in 'xyz' for char in sequence]) or len(sequence) != 3:
        raise ValueError(f'Invalid or unsupported euler angle sequence {sequence}.')
    T1 = hrotation3(sequence[0], euler_angles[0])
    T2 = hrotation3(sequence[1], euler_angles[1])
    T3 = hrotation3(sequence[2], euler_angles[2])
    return T1 * T2 * T3
def hrotation3(axis, angle):
    """
    Return a transformation matrix in 3D homogeneous coordinates that represents the rotation by `angle` around `axis`.
    
    :param axis: is the axis of rotation. It should be one of ['x', 'y', 'z', 'n', 'o', 'a'].
    :type axis: string
    ...
    :param angle: is the angle of rotation.
    :type angle: sympy expression, sympy symbol or a number
    ...
    raise ValueError: if an invalid axis value is received.
    ...
    :return: the rotation matrix
    """
    if axis in ['x', 'n']:
        return sp.Matrix([
            [1, 0, 0, 0],
            [0, sp.cos(angle), -sp.sin(angle), 0], 
            [0, sp.sin(angle), sp.cos(angle), 0],
            [0, 0, 0, 1]
        ])
    elif axis in ['y', 'o']:
        return sp.Matrix([
            [sp.cos(angle), 0, sp.sin(angle), 0],
            [0, 1, 0, 0],
            [-sp.sin(angle), 0, sp.cos(angle), 0],
            [0, 0, 0, 1]
        ])
    elif axis in ['z', 'a']:
        return sp.Matrix([
            [sp.cos(angle), -sp.sin(angle), 0, 0], 
            [sp.sin(angle), sp.cos(angle), 0, 0], 
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
    else:
        raise ValueError(f'Expected one of [x, y, z, n, o, a] but received {axis}.') 

# DH Parameter Layout:
# ['r', d, a, alpha] for revolute joints
# ['p', theta, a, alpha] for prismatic joints
# ['f', d, theta, a, alpha] for fixed joints
def htranslation3(x=0, y=0, z=0):
    """Return transformation matrix in 3D homogeneous coordinates with embedded translation."""
    return sp.Matrix([[1, 0, 0, x],
                      [0, 1, 0, y],
                      [0, 0, 1, z],
                      [0, 0, 0, 1]])

def xml_string(DH_Params, scale=1):

    outstring = "<!-- DH Parameters \n"
    for DH_param in DH_Params:
        outstring = outstring + "{}\n".format(str(DH_param)[1:-1])
    outstring = outstring + "-->\n"

    transforms = joint_transforms(DH_Params)

    frames = joint_frames(transforms)

    outstring = outstring + "<robot name='robot'>\n"

    outstring = outstring + "\t<material name='blue'>\n\t\t<color rgba='0.6 0.6 0.6 1'/>\n\t</material>\n"
    outstring = outstring + "\t<material name='red'>\n\t\t<color rgba='1 1 1 1'/>\n\t</material>\n"
    for i in range(len(transforms) - 1):

        el = transforms[i]
        fr = frames[i]
        

        # We need to create a cylinder to represent the joint
        # If the index is not zero, connect it to the previous link
        # And a joint to connect it to the link
        # And a box to connect the joints

        rpy = euler_angles(fr)
        
        outstring = outstring + "\t<link name='a{}'>\n".format(i)
        outstring = outstring + "\t\t<visual>\n"
        outstring = outstring + "\t\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(rpy[0], rpy[1], rpy[2], el[0,3], el[1,3], el[2,3])
        outstring = outstring + "\t\t\t<geometry>\n"
        outstring = outstring + "\t\t\t\t<cylinder length='0.7' radius='0.4'/>\n"
        outstring = outstring + "\t\t\t</geometry>\n"
        outstring = outstring + "\t\t\t<material name='blue'/>\n"
        outstring = outstring + "\t\t</visual>\n"
        if (DH_Params[i][0] == 'p'and False): 
           
            a = np.concatenate((fr[0:3,0:3],np.zeros(shape = (1,3), dtype = 'int')),axis=0)
            rotm = np.concatenate((a, np.array([[0],[0],[0],[1]]) ),axis=1)
            rotms = rotation_matrix_from_euler_angles(sp.Matrix([[rpy[0]],[rpy[1]],[rpy[2]]]),'XYZ')
            posm = htranslation3( el[0,3], el[1,3], el[2,3])
            ofsm = rotms * htranslation3(0,0,2)
            posm[:,3] = posm[:,3] + ofsm[:,3]
          
            tr= posm*rotms
            rpy = euler_angles(np.array((posm*rotm).evalf(),dtype=float))
            outstring = outstring + "\t\t<visual>\n"
            outstring = outstring + "\t\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(rpy[0], rpy[1], rpy[2], tr[0,3], tr[1,3], tr[2,3])
            outstring = outstring + "\t\t\t<geometry>\n"
            outstring = outstring + "\t\t\t\t<cylinder length='4' radius='0.3'/>\n"
            outstring = outstring + "\t\t\t</geometry>\n"
            outstring = outstring + "\t\t</visual>\n"
        outstring = outstring + "\t</link>\n"

        # If not on the first transformation, fix the cylinder to the previous link
        if(i != 0):
            outstring = outstring + "\t<joint name='fix_a{}_to_l{}' type='fixed'>\n".format(i, i-1)
            outstring = outstring + "\t\t<parent link='l{}'/>\n".format(i-1)
            outstring = outstring + "\t\t<child link='a{}'/>\n".format(i)
            outstring = outstring + "\t\t<origin rpy='0 0 0' xyz='0 0 0'/>\n"
            outstring = outstring + "\t</joint>\n"

        # Add a cylinder that goes from the current origin to the next one
        origins_vector = transforms[i + 1][0:3,3]

        origins_vector_norm = np.linalg.norm(origins_vector)

        cylinder_origin = origins_vector/2

        rpy = [0, 0, 0]
        if (origins_vector_norm != 0.0):
            origins_vector_unit = origins_vector/origins_vector_norm

            axis = np.cross(origins_vector, np.array([0, 0, -1]))
            axis_norm = np.linalg.norm(axis)
            if (axis_norm != 0.0):
                axis = axis/np.linalg.norm(axis)

            angle = np.arccos(origins_vector_unit @ np.array([0, 0, 1]))

            print("axis is {}".format(axis))
            print("angle is {}". format(angle))


            print(angle * axis )
            s = angle * axis
            rpy = R.from_rotvec(angle * axis).as_euler('XYZ') # moze ama i ne treba 
            rpy = angle * axis

        outstring = outstring + "\t<link name='l{}'>\n".format(i)
        outstring = outstring + "\t\t<visual>\n"
        outstring = outstring + "\t\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(rpy[0], rpy[1], rpy[2], cylinder_origin[0], cylinder_origin[1], cylinder_origin[2])
        outstring = outstring + "\t\t\t<geometry>\n"
        outstring = outstring + "\t\t\t\t<cylinder length='{}' radius='0.3'/>\n".format(origins_vector_norm) 
        outstring = outstring + "\t\t\t</geometry>\n"
        outstring = outstring + "\t\t\t<material name='red'/>\n"
        outstring = outstring + "\t\t</visual>\n"
        if (DH_Params[i][0] == 'p'): 
            origins_vector = transforms[i][0:3,3]/2
            rpy = euler_angles(fr) 
            a = np.concatenate((fr[0:3,0:3],np.zeros(shape = (1,3), dtype = 'int')),axis=0)
            rotm = np.concatenate((a, np.array([[0],[0],[0],[1]]) ),axis=1)
            rotms = rotation_matrix_from_euler_angles(sp.Matrix([[rpy[0]],[rpy[1]],[rpy[2]]]),'XYZ')
            posm = htranslation3( fr[0,3], fr[1,3], fr[2,3])
            ofsm = rotms * htranslation3(0,0,-1.7)
            posm[:,3] = posm[:,3] - ofsm[:,3]
            print(DH_Params[i-1][2])
            print(ofsm*rotms)
            tr= ofsm*rotms
            
            outstring = outstring + "\t\t<visual>\n"
            outstring = outstring + "\t\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(rpy[0], rpy[1], rpy[2],tr[0,3], tr[1,3], tr[2,3])
            outstring = outstring + "\t\t\t<geometry>\n"
            outstring = outstring + "\t\t\t\t<cylinder length='4' radius='0.3'/>\n"
            outstring = outstring + "\t\t\t</geometry>\n"
            outstring = outstring + "\t\t</visual>\n"
        outstring = outstring + "\t</link>\n"

        # Add the actual joint between the cylinder and link

        jointType = ""

        if (DH_Params[i][0] == 'r'):
            jointType = "revolute"
        elif (DH_Params[i][0] == 'p'):
            jointType = "prismatic"
        else:
            jointType = "fixed"

        outstring = outstring + "\t<joint name='move_l{}_from_a{}' type='{}'>\n".format(i, i, jointType)
        outstring = outstring + "\t\t<parent link='a{}'/>\n".format(i)
        outstring = outstring + "\t\t<child link='l{}'/>\n".format(i)
        outstring = outstring + "\t\t<axis xyz='{} {} {}'/>\n".format(fr[0,2], fr[1,2], fr[2,2])
        outstring = outstring + "\t\t<origin rpy='0 0 0' xyz='{} {} {}'/>\n".format(el[0,3], el[1,3], el[2,3])
        if (DH_Params[i][0] == 'p'):
            outstring = outstring + "\t\t<limit effort='12' lower='0.4' upper='4' velocity='2.6100'/>\n" 
        else:  
            outstring = outstring + "\t\t<limit effort='12' lower='-2.9671' upper='2.9671' velocity='2.6100'/>\n" 
         
        outstring = outstring + "\t</joint>\n" 

    outstring = outstring + "</robot>\n"

    return outstring

In [118]:
f = open("outfile.urdf", "w")

DH_Params = []
DH_Params.append(['r', 1, 2, 0])
DH_Params.append(['r', 1, 2.00, pi/2])
DH_Params.append(['p', 0, 3, 0])
DH_Params.append(['f', 2, 0, 2, 0])
DH_Params.append(['r', 0, 2, 0])

f.write(xml_string(DH_Params))

f.close()


axis is [-0.  1.  0.]
angle is 1.1071487177940904
[-0.          1.10714872  0.        ]
axis is [-0.  1.  0.]
angle is 1.1071487177940904
[-0.          1.10714872  0.        ]
axis is [-0.  1.  0.]
angle is 1.5707963267948966
[-0.          1.57079633  0.        ]
2.0
Matrix([[1, 0, 0, 0], [0, -1.00000000000000, -1.22464679914735e-16, 1.70000000000000], [0, 1.22464679914735e-16, -1.00000000000000, -1.04094977927525e-16], [0, 0, 0, 1]])
axis is [-0.  1.  0.]
angle is 0.7853981633974484
[-0.          0.78539816  0.        ]
axis is [-0.  1.  0.]
angle is 1.5707963267948966
[-0.          1.57079633  0.        ]


C:\Users\vase_\AppData\Local\Temp\ipykernel_5044\1247573994.py:223: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  rpy = R.from_rotvec(angle * axis).as_euler('XYZ') # moze ama i ne treba
